In [30]:
import cv2
import numpy as np
import glob
import json
import os

number_of_squares_x = 11
number_of_internal_corners_x = number_of_squares_x - 1
number_of_squares_y = 8
number_of_internal_corners_y = number_of_squares_y - 1
square_size = 0.023 # in meters

In [31]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(9,6,0)
objp = np.zeros((number_of_internal_corners_x * number_of_internal_corners_y,3), np.float32)
objp[:,:2] = np.mgrid[0:number_of_internal_corners_x,0:number_of_internal_corners_y].T.reshape(-1,2)

In [32]:
def intrinsic_calibration(camera_name):
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    images = glob.glob('screen_shots/*.png')

    shape = (0,0)

    effective_chessboard_count = 0
    total_chessboard_count = 0

    for fname in images:
        fname_parts = fname.split('/')[1].split('_')
        if fname_parts[0] != camera_name:
            continue
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        shape = gray.shape[::-1]

        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (number_of_internal_corners_x,number_of_internal_corners_y), None)
        
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            # print(f'{fname} pattern found')
            effective_chessboard_count += 1
        else:
            # print(f'{fname} pattern not found')
            pass
            
        total_chessboard_count += 1

    if (effective_chessboard_count < 10):
        raise ValueError(f'Not enough effective chessboard found: {effective_chessboard_count}/{total_chessboard_count}')

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, shape, None, None)

    print(f'Camera matrix: {mtx}')
    print(f'Distortion coefficients: {dist}')
    print(f'Effective Chessboard: {effective_chessboard_count}/{total_chessboard_count}')

    with open(f'results/calibration_{camera_name}.json', 'w') as f:
        json.dump({'mtx': mtx.tolist(), 'dist': dist.tolist()}, f)

In [33]:
cameras = ['cam1', 'cam2', 'cam3', 'cam4', 'cam5']
for camera in cameras:
    intrinsic_calibration(camera)

Camera matrix: [[852.92006731   0.         319.04352479]
 [  0.         860.33372762 240.28107357]
 [  0.           0.           1.        ]]
Distortion coefficients: [[ 8.58182086e-01 -1.11999357e+01  6.25391411e-02 -2.25347230e-02
   3.68946777e+01]]
Effective Chessboard: 17/28
Camera matrix: [[758.20435017   0.         304.88067183]
 [  0.         798.39907757 154.93900572]
 [  0.           0.           1.        ]]
Distortion coefficients: [[ 1.29269789e-01  2.60402407e+00 -4.63226859e-02  6.18630846e-03
  -8.40821248e+00]]
Effective Chessboard: 21/28
Camera matrix: [[623.21217981   0.         325.81564993]
 [  0.         604.00086592 247.35197019]
 [  0.           0.           1.        ]]
Distortion coefficients: [[ 1.31395540e+00 -8.02159156e+00 -1.50516298e-02  1.41082988e-02
  -1.80294497e+01]]
Effective Chessboard: 27/28
Camera matrix: [[750.83807107   0.         327.92656874]
 [  0.         763.10662395 168.33279003]
 [  0.           0.           1.        ]]
Distortion coef